# Extraction des questions ouvertes

Ce notebook a pour but de développer des fonctions nécessaires et utilisables sur les questions fermées des trois questionnaires des Assises de l'enseignement supérieur organisé par la région Île-De-France. 

Plusieurs types de représentations des textes peuvent être considérés:
- Tf-idf

- Word-Embeddings

De même, plusieurs types de fonctions sont à considérer, suivant différents buts:



In [1]:
#Loading packages
import numpy as np
import pandas as pd
import sys

import pandas as pd
import numpy as np
import datetime
import os


import string
import spacy 
import nltk
from normalise import normalise
from nltk.tokenize import word_tokenize

from sklearn.feature_extraction.text import TfidfVectorizer

import spacy.lang.fr
import spacy

global spacy_nlp
spacy_nlp = spacy.load('fr_core_news_sm')
import normalise

STOPWORDS = spacy.lang.fr.stop_words.STOP_WORDS

import string


from sklearn.decomposition import PCA
from sklearn.decomposition import TruncatedSVD
from sklearn.manifold import TSNE


import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.cluster import KMeans
from sklearn.feature_selection import chi2

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from textwrap import wrap

from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.datasets import fetch_20newsgroups
from sklearn.decomposition import NMF, LatentDirichletAllocation

from openpyxl import load_workbook

C:\Users\adrie\Anaconda3\lib\site-packages\sklearn\base.py:306: UserWarning: Trying to unpickle estimator LabelPropagation from version 0.18 when using version 0.21.3. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)
C:\Users\adrie\Anaconda3\lib\site-packages\statsmodels\tools\_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [2]:
pd.set_option('display.max_columns', 50)
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_colwidth', 500)

## Importing DATA

### Contributions

Création d'un dictionnaire pour faciliter l'affichage :

In [3]:
questionnaires = {
    'chercheurs, enseignants-chercheurs et ingénieurs.xlsx':'Questionnaire à destination des chercheurs, enseignants-chercheurs et ingénieurs',
    'enseignants ou personnels encadrants des campus.xlsx':'Questionnaire à destination des enseignants ou personnels encadrants des campus',
    'étudiants.xlsx':'Questionnaire à destination des étudiants'
}

In [4]:
filenames = list(questionnaires.keys())
themes = list(questionnaires.values())

In [5]:
filepaths = [os.path.join('Data/',filename) for filename in filenames]

Définition de la colonne `Created`, de type `date`

In [6]:
col_date = ['Created']

In [7]:
df_list = [pd.read_excel(filepath, parse_dates=col_date) for filepath in filepaths]

Colonnes communes à tous les datasets (tous les thèmes):

Cela permettra d'extraire les questions de chaque thème, puisque les seules colonnes qui ne sont pas en commun à tous les thèmes sont les colonnes qui correspondent aux questions.

In [8]:
col_common = ['Created']

On affiche des informations générales sur les thèmes:

In [9]:
df_infos = pd.DataFrame({
     'Questionnaire': themes,
     'nombre de sondés': [df.shape[0] for df in df_list],
     'nombre de questions': [sum(~df.columns.isin(col_common)) for df in df_list]
    })
df_infos

,Questionnaire,nombre de sondés,nombre de questions
0,"Questionnaire à destination des chercheurs, enseignants-chercheurs et ingénieurs",242,12
1,Questionnaire à destination des enseignants ou personnels encadrants des campus,37,22
2,Questionnaire à destination des étudiants,623,21


### Definitions des questions

In [10]:
df_def_questions=pd.read_csv('Data/utilities/df_def_questions.csv')

In [11]:
df_def_questions['type de la sous question'].unique()

array(['cmq', 'cq', 'oq', 'mixed_tq_insatisfaisant',
       'multi_mixed_cq_autre'], dtype=object)

In [12]:
df_def_open_questions=df_def_questions[df_def_questions['type de la sous question'].isin(['mixed_tq_insatisfaisant', 'oq', 'multi_mixed_cq_autre'])].reset_index(drop=True)

In [14]:
df_def_open_questions.reset_index(drop=True, inplace=True)

In [15]:
df_def_open_questions

,questionnaire,numero questionnaire,numéro de la question,titre de la question,type de la sous question,sous question,sq_titre_viz
0,"Questionnaire à destination des chercheurs, enseignants-chercheurs et ingénieurs",0,Q3,"Voici la liste des différents Domaines d’intérêt majeur de recherche : Math'innov, Matériaux anciens et patrimoniaux (MAP), Science et ingénierie en région Île-de-France pour les technologies quantiques (SIRTEQ), Islam en Île-de-France, Astrophysique et conditions d'apparition de la vie (ACAV+), Ruptures en sciences des solides poreux (Respore), Thérapie génique, Un monde, une seule sa...",oq,"Voici la liste des différents Domaines d’intérêt majeur de recherche : Math'innov, Matériaux anciens et patrimoniaux (MAP), Science et ingénierie en région Île-de-France pour les technologies quantiques (SIRTEQ), Islam en Île-de-France, Astrophysique et conditions d'apparition de la vie (ACAV+), Ruptures en sciences des solides poreux (Respore), Thérapie génique, Un monde, une seule santé (One Health), Empowering Life sCiences with Innovative Technologies – Technologies innovantes pour les s...",Quel ou quels DIM vous paraissent nécessaire d’ajouter à la liste ?
1,"Questionnaire à destination des chercheurs, enseignants-chercheurs et ingénieurs",0,Q4,Comment évaluez-vous le ou les dispositifs que vous connaissez ?,mixed_tq_insatisfaisant,Les DIM-domaines d’intérêt majeur de recherche,Les DIM-domaines d’intérêt majeur de recherche
2,"Questionnaire à destination des chercheurs, enseignants-chercheurs et ingénieurs",0,Q4,Comment évaluez-vous le ou les dispositifs que vous connaissez ?,mixed_tq_insatisfaisant,SESAME (les équipements pour les laboratoires),SESAME (les équipements pour les laboratoires)
3,"Questionnaire à destination des chercheurs, enseignants-chercheurs et ingénieurs",0,Q4,Comment évaluez-vous le ou les dispositifs que vous connaissez ?,mixed_tq_insatisfaisant,Les Chaires d’excellence internationale Blaise Pascal,Les Chaires d’excellence internationale Blaise Pascal
4,"Questionnaire à destination des chercheurs, enseignants-chercheurs et ingénieurs",0,Q4,Comment évaluez-vous le ou les dispositifs que vous connaissez ?,mixed_tq_insatisfaisant,Les bourses doctorales Paris Region PhD,Les bourses doctorales Paris Region PhD
5,"Questionnaire à destination des chercheurs, enseignants-chercheurs et ingénieurs",0,Q4,Comment évaluez-vous le ou les dispositifs que vous connaissez ?,mixed_tq_insatisfaisant,SESAME filière PIA,SESAME filière PIA
6,"Questionnaire à destination des chercheurs, enseignants-chercheurs et ingénieurs",0,Q4,Comment évaluez-vous le ou les dispositifs que vous connaissez ?,mixed_tq_insatisfaisant,Les bourses postdoctorales du programme européen Paris Region fellowship COFUND,Les bourses postdoctorales du programme européen Paris Region fellowship COFUND
7,"Questionnaire à destination des chercheurs, enseignants-chercheurs et ingénieurs",0,Q4,Comment évaluez-vous le ou les dispositifs que vous connaissez ?,mixed_tq_insatisfaisant,Le dispositif ACCESS de soutien à la mobilité des étudiants et des chercheurs internationaux,Le dispositif ACCESS de soutien à la mobilité des étudiants et des chercheurs internationaux
8,"Questionnaire à destination des chercheurs, enseignants-chercheurs et ingénieurs",0,Q5,"Pour développer vos partenariats en matière de R&D avec des entreprises en Ile-de-France, en particulier des PME/ETI, quels freins rencontrez-vous ?",multi_mixed_cq_autre,"Pour développer vos partenariats en matière de R&D avec des entreprises en Ile-de-France, en particulier des PME/ETI, quels freins rencontrez-vous ?","Pour développer vos partenariats en matière de R&D avec des entreprises en Ile-de-France, en particulier des PME/ETI, quels freins rencontrez-vous ?"
9,"Questionnaire à destination des chercheurs, enseignants-chercheurs et ingénieurs",0,Q6,Avez-vous des idées sur les évolutions à apporter aux dispositifs d’intervention de la Région en matière de recherche ?,oq,[Boîte à idées] Avez-v

### Saving open questions

In [63]:
def extract_open(df_list, themes, df_def):
    for n_questionnaire in range(len(themes)):

        #On selectionne seulement les colonnes du questionnaire considéré:
        df_def_temp=df_def[df_def['numero questionnaire']==n_questionnaire].reset_index(drop=True)

        df_def_temp=df_def_temp.reset_index(drop=True)
        #On affiche l'histogramme des réponses pour chaque question:    
        for index, row in df_def_temp.iterrows():              
            #tfidf_matrix: 
            col_name=row['sous question']
            df_temp=pd.DataFrame()
            df_temp['reponses']=df_list[n_questionnaire][col_name].copy()
            #Dropping nans:
            df_temp=df_temp.dropna()

            df_temp=df_temp[df_temp['reponses'] != 'plutôt satisfaisant']
            df_temp=df_temp[df_temp['reponses']!= 'très satisfaisant']
            df_temp.reset_index(drop=True, inplace=True)
            
            titre_question = str(row['sq_titre_viz']).replace("[Boîte à idées]", "")
            
            
            path = 'Rendu_final/Extractions/'+str(n_questionnaire)+'_'+str(row['type de la sous question'])+'.xlsx'
            
            
            with pd.ExcelWriter(path, engine='openpyxl') as writer: 
                
                book = load_workbook(path)
                writer.book = book
                writer.sheets = dict((ws.title, ws) for ws in book.worksheets)
                df_temp.to_excel(writer, str(row['numéro de la question'])+'_'+titre_question[:20], encoding="utf-8")    
                writer.save() 
            
    return None

In [64]:
extract_open(df_list, themes, df_def_open_questions)